In [1]:
import numpy as np
import cv2 as cv
import math

In [70]:
img = cv.imread('hough.jpg',0)
[rows,cols] = img.shape

# Padding

In [71]:
def padding(img):
    [rows,cols] = img.shape
    listofzeros = [0] * (cols+2)
    image_pad = list()
    image_pad.append(listofzeros)
    for i in img:
        i = list(i)
        i.insert(0,0)
        i.insert(len(i)+1,0)
        image_pad.append(i)
    image_pad.append(listofzeros)
    image_pad = np.asarray(image_pad)
    return image_pad

# Convolution

In [72]:
def convolution(image,kernel):
    [rows,cols] = image.shape
    image_pad = padding(image)
    conv_img = [[0] * (int)(cols) for _ in range((int)(rows))]
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            s=0
            for k in range(3):
                for l in range(3):
                    s+=image_pad[i-1+k,j-1+l]*kernel[k,l]
            conv_img[i-1][j-1] = s
    return np.asarray(conv_img)

# Edge detection

In [73]:
def edges(conv_img,threshold):
    for i in range(conv_img.shape[0]):
        for j in range(conv_img.shape[1]):
            if conv_img[i,j]>threshold:
                conv_img[i,j] = 255
            else:
                conv_img[i,j] = 0
    return conv_img

In [74]:
x_sob = [[1, 0, -1], 
         [2, 0, -2], 
         [1, 0, -1]]
y_sob = [[1, 2, 1], 
         [0, 0, 0], 
         [-1, -2, -1]]
x_sob = np.asarray(x_sob)
y_sob = np.asarray(y_sob)

In [75]:
conv_img_x = convolution(img,x_sob)
conv_img_x = edges(conv_img_x,200)
cv.imwrite('x_sob.png',conv_img_x)

True

In [76]:
conv_img_y = convolution(img,y_sob)
conv_img_y = edges(conv_img_y,70)
cv.imwrite('y_sob.png',conv_img_y)

True

In [77]:
conv_img = np.empty([rows,cols])
for i in range(rows):
    for j in range(cols):
        conv_img[i,j] = math.sqrt(conv_img_x[i,j]**2 + conv_img_y[i,j]**2)

In [78]:
cv.imwrite('edges.png',conv_img)

True

# Hough transform

In [188]:
def find_rho(x,y,theta,diag):
    return int((y*math.cos(math.radians(theta)) + x*math.sin(math.radians(theta))+diag))

# Accumulator matrix

In [189]:
diag = math.sqrt(rows**2 + cols**2)

In [190]:
def accumulator_matrix(conv):
    accumulator = np.zeros((180,2*int(diag)-1))
    for i in range(conv.shape[0]):
        for j in range(conv.shape[1]):
            if conv[i,j] == 255:
                for k in range(-90,90):
                    rho_val = find_rho(i,j,k,diag)
                    theta = k + 90
                    accumulator[theta,rho_val] = accumulator[theta,rho_val] + 1
    return accumulator

In [191]:
def accumulator_matrix_y(conv):
    accumulator = np.zeros((160,2*int(diag)-1))
    for i in range(conv.shape[0]):
        for j in range(conv.shape[1]):
            if conv[i,j] == 255:
                for k in range(-80,80):
                    rho_val = find_rho(i,j,k,diag)
                    theta = k + 80
                    accumulator[theta,rho_val] = accumulator[theta,rho_val] + 1
    return accumulator

In [192]:
accumulator_x = accumulator_matrix(conv_img_x)
cv.imwrite('accumulator_vertical.png',accumulator_x)

True

In [193]:
accumulator_x.shape

(180, 1637)

In [194]:
accumulator_y = accumulator_matrix_y(conv_img_y)
cv.imwrite('accumulator_diagonal.png',accumulator_y)

True

In [195]:
accumulator_y.shape

(160, 1637)

# Finding peaks in the sinusoidal curve

In [196]:
def max_indices(arr, k):
    '''
    Returns the indices of the k first largest elements of arr
    (in descending order in values)
    '''
    assert k <= arr.size, 'k should be smaller or equal to the array size'
    arr_ = arr.astype(float)  # make a copy of arr
    max_idxs = []
    for _ in range(k):
        max_element = np.max(arr_)
        if np.isinf(max_element):
            break
        else:
            idx = np.where(arr_ == max_element)
        max_idxs.append(idx)
        arr_[idx] = -np.inf
    return np.asarray(max_idxs)

In [197]:
def points(indices):
    k = []
    for i in range(indices.shape[0]):
        n = indices[i][0].shape[0]
        if n>1:
            for j in range(n):
                k.append([indices[i][0][j], indices[i][1][j]])
        else:
            k.append([indices[i][0][0], indices[i][1][0]])
    return np.asarray(k)

In [198]:
indices_x = max_indices(accumulator_x,40)
indices_x = points(indices_x)
indices_x.shape

(43, 2)

In [216]:
indices_y = max_indices(accumulator_y,35)
indices_y = points(indices_y)
indices_y.shape

(43, 2)

# Plotting lines

In [200]:
def plotting_lines(conv,indices,r,g,b):
    img = cv.imread('hough.jpg')
    coord = []
    for i in range(rows):
        for j in range(cols):
            if conv[i,j] == 255:
                for k in range(indices.shape[0]):
                    r = find_rho(i,j,indices[k,0]-90,diag)
                    if r == indices[k,1]:
                        coord.append([i,j])
                        img[i,j] = (r,g,b)
    coord = np.asarray(coord)
    return img

In [212]:
def plotting_lines_y(conv,indices,r,g,b):
    img = cv.imread('hough.jpg')
    res_img = img
    coord = []
    for i in range(rows):
        for j in range(cols):
            if conv[i,j] == 255:
                for k in range(indices.shape[0]):
                    r = find_rho(i,j,indices[k,0]-80,diag)
                    if r == indices[k,1]:
                        coord.append([i,j])
                        res_img[i,j] = (r,g,b)
    coord = np.asarray(coord)
    return res_img

In [218]:
vertical_lines = plotting_lines(conv_img_x,indices_x,0,255,255)
cv.imwrite('red_lines.png',vertical_lines)

True

In [219]:
diagonal_lines = plotting_lines_y(conv_img_y,indices_y,0,255,200)
cv.imwrite('blue_lines.png',diagonal_lines)

True